## Gemini 실행 설정

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 6.5 MB/s eta 0:00:00


### GCP 환경설정 및 로그인

In [2]:
import os

PROJECT_ID = "ai-hangsik" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USE_VERTEX_AI = True #@param {type:"boolean"}
MODEL = "gemini-2.5-flash" #@param {type:"string"}

In [ ]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project {PROJECT_ID}

### Vertex AI Client 실행

In [4]:
import base64
from IPython.display import Image, display, Markdown

from google import genai
from google.genai import types
from google.genai.types import HttpOptions

client = genai.Client(
    vertexai=USE_VERTEX_AI,
    project=PROJECT_ID,
    location=REGION,)

### Gemini 실행

#### pydantic 형태 제어구문

In [14]:
from pydantic import BaseModel

class Recipe_pydantic(BaseModel):
    name: str
    description: str
    ingredients: list[str]


In [15]:
response = client.models.generate_content(
      model = MODEL,

      contents = types.Content(
        role="user",
        parts=[
          types.Part.from_text(text="""한국의 대표적인 음식과 필요한 재료를 알려주세요.  """)
        ]
      ),

      # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
      config = types.GenerateContentConfig(
        system_instruction = "당신은 음식에 대한 전문가입니다. 쉽게 설명 해주세요.",
        response_schema=Recipe_pydantic,
        response_mime_type="application/json",
      ),
  )

display(Markdown(response.text))


{
  "name": "비빔밥 (Bibimbap)",
  "description": "비빔밥은 따뜻한 밥 위에 다양한 색깔의 신선한 채소, 고기, 그리고 계란 프라이를 보기 좋게 올린 한국의 대표적인 건강 음식입니다. 모든 재료를 매콤달콤한 고추장과 함께 비벼서 먹습니다. 한 그릇 안에 여러 가지 맛과 영양이 균형 있게 담겨 있어 외국인들에게도 인기가 많습니다.",
  "ingredients": [
    "밥 (Cooked rice)",
    "다양한 나물 (시금치, 콩나물, 고사리 등) (Various seasoned vegetables like spinach, bean sprouts, fernbrake)",
    "소고기 또는 닭고기 (Beef or chicken)",
    "계란 (Egg)",
    "고추장 (Korean chili paste)",
    "참기름 (Sesame oil)",
    "간장 (Soy sauce)",
    "마늘 (Garlic)",
    "깨소금 (Sesame seeds)"
  ]
}

#### JSON 형태 제어구문

In [20]:
Recipe_JSON = {
    "type": "OBJECT",
    "properties": {
        "name": {"type": "STRING"},
        "description": {"type": "STRING"},
        "ingredients": {
            "type": "ARRAY",
            "items": {"type": "STRING"},
        },
    }
}

In [21]:
response = client.models.generate_content(
      model = MODEL,

      contents = types.Content(
        role="user",
        parts=[
          types.Part.from_text(text="""한국의 대표적인 음식과 필요한 재료를 알려주세요.  """)
        ]
      ),

      # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
      config = types.GenerateContentConfig(
        system_instruction = "당신은 음식에 대한 전문가입니다. 쉽게 설명 해주세요.",
        response_schema=Recipe_JSON,
        response_mime_type="application/json",
      ),
  )

display(Markdown(response.text))

{"name":"김치찌개","description":"한국인의 밥상에서 빠질 수 없는, 김치와 돼지고기가 어우러져 얼큰하고 깊은 맛을 내는 대표적인 찌개 요리입니다.","ingredients":["김치","돼지고기","두부","파","양파","고춧가루","다진 마늘","국간장 또는 새우젓","멸치 육수 또는 물"]}

## End of Document